# 1. Agentes con Memoria Conversacional en LangChain

## Objetivos de Aprendizaje
- Comprender por qué la memoria es crucial para crear agentes conversacionales efectivos.
- Aprender a gestionar el historial de una conversación (`chat_history`) con el `AgentExecutor`.
- Implementar un agente que recuerde interacciones pasadas para responder preguntas de seguimiento.
- Entender cómo LangChain pasa el contexto de la conversación al LLM.

## ¿Qué es la Memoria y Por Qué es Importante?

Por defecto, los LLMs y los agentes que hemos construido hasta ahora **no tienen estado (stateless)**. Cada vez que los invocamos, procesan la solicitud como si fuera la primera vez que interactúan con nosotros. No tienen recuerdo de preguntas o respuestas anteriores.

Esto es una gran limitación para crear asistentes o chatbots útiles. Un usuario espera poder hacer preguntas de seguimiento, referirse a información mencionada previamente y tener una conversación fluida. 

La **memoria** es el mecanismo que permite a un agente recordar interacciones pasadas. LangChain facilita enormemente la gestión de esta memoria. La forma más común de memoria es el **historial de chat (chat history)**, donde simplemente guardamos la lista de todos los mensajes de la conversación.

En este notebook, veremos cómo añadir esta capacidad a nuestro agente de LangChain.

### 1. Instalación y Configuración

In [ ]:
!pip install langchain langchain-openai openai wikipedia -q

In [2]:
import os
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang("es")

# Configuración del LLM
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("GITHUB_BASE_URL"),
        openai_api_key=os.environ.get("GITHUB_TOKEN"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

✅ LLM de LangChain configurado.


### 2. Herramientas y Agente (Sin Cambios)

La definición de las herramientas y la creación del agente son exactamente las mismas que en el notebook anterior. La magia de la memoria no está en la definición del agente, sino en **cómo lo ejecutamos**.

In [3]:
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain import hub

@tool
def get_wikipedia_summary(query: str) -> str:
    """Busca en Wikipedia un tema y devuelve un resumen de 2 frases. Útil para obtener información sobre personas, lugares o conceptos."""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Ocurrió un error: {e}"

tools = [get_wikipedia_summary]

# Usamos el mismo prompt de la comunidad que ya está preparado para manejar historial
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ Agente y herramientas listos.")

✅ Agente y herramientas listos.


### 3. Gestionando la Memoria Conversacional

Para que el agente recuerde, necesitamos hacer dos cosas:

1.  **Mantener un historial**: Crearemos una lista llamada `chat_history` para almacenar los mensajes.
2.  **Pasar el historial en cada llamada**: El `AgentExecutor` acepta un parámetro `chat_history`. LangChain se encarga de formatear esta lista y añadirla al prompt que se envía al LLM.

El formato del historial es una lista de objetos `BaseMessage` de LangChain. Los más comunes son `HumanMessage` (para el usuario) y `AIMessage` (para la respuesta del agente).

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

# Iniciamos el historial de chat como una lista vacía
chat_history = []

#### Primera Interacción: Sin Historial

In [8]:
query1 = "Háblame del planeta Saturno"

response1 = agent_executor.invoke({
    "input": query1,
    "chat_history": chat_history
})

print(f"Respuesta 1: {response1['output']}")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `get_wikipedia_summary` with `{'query': 'Saturno (planeta)'}`


Saturno es el sexto planeta del sistema solar contando desde el Sol, el segundo en tamaño y masa después de Júpiter y el único con un sistema de anillos visible desde la Tierra. Su nombre proviene del dios romano Saturno.Saturno es el sexto planeta del sistema solar, conocido por ser el segundo en tamaño y masa después de Júpiter. Es famoso por su sistema de anillos, que es visible desde la Tierra, y su nombre proviene del dios romano Saturno.

> Finished chain.
Respuesta 1: Saturno es el sexto planeta del sistema solar, conocido por ser el segundo en tamaño y masa después de Júpiter. Es famoso por su sistema de anillos, que es visible desde la Tierra, y su nombre proviene del dios romano Saturno.


Ahora, actualizamos manualmente nuestro historial con la pregunta del usuario y la respuesta del agente.

In [9]:
chat_history.append(HumanMessage(content=query1))
chat_history.append(AIMessage(content=response1["output"]))

print("Historial actualizado.")

Historial actualizado.


#### Segunda Interacción: Con Historial

Ahora hacemos una pregunta de seguimiento. Fíjate que no mencionamos "Saturno", simplemente preguntamos "¿de qué están hechos sus anillos?"

In [11]:
query2 = "¿Y de qué están hechos sus anillos?"

response2 = agent_executor.invoke({
    "input": query2,
    "chat_history": chat_history
})

print(f"Respuesta 2: {response2['output']}")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Los anillos de Saturno están compuestos principalmente de partículas de hielo de agua, con pequeñas cantidades de roca y polvo. Estas partículas varían en tamaño, desde diminutos granos de polvo hasta fragmentos tan grandes como una casa. Los anillos son extremadamente delgados en comparación con su extensión, lo que los hace una de las características más fascinantes del sistema solar.

> Finished chain.
Respuesta 2: Los anillos de Saturno están compuestos principalmente de partículas de hielo de agua, con pequeñas cantidades de roca y polvo. Estas partículas varían en tamaño, desde diminutos granos de polvo hasta fragmentos tan grandes como una casa. Los anillos son extremadamente delgados en comparación con su extensión, lo que los hace una de las características más fascinantes del sistema solar.


¡Funcionó! El agente entendió que "sus anillos" se refería a los anillos de Saturno, porque la conversación anterior estaba en su contexto. El `verbose=True` nos muestra que el agente decidió buscar en Wikipedia "anillos de Saturno", combinando la nueva pregunta con el historial.

## Conclusiones

Añadir memoria a un agente de LangChain es sorprendentemente sencillo, pero increíblemente poderoso. Simplemente manteniendo una lista del historial de chat y pasándola en cada invocación, transformamos un agente de una sola respuesta en un verdadero **asistente conversacional**.

La clave es que los componentes de LangChain (`AgentExecutor`, los prompts de `hub`) ya están diseñados para buscar y utilizar la variable `chat_history` si se proporciona.

**Limitaciones:**
- **Gestión Manual**: En este ejemplo, actualizamos la lista `chat_history` manualmente. Para una aplicación real, querríamos encapsular esto en una clase o función.
- **Tamaño del Contexto**: Enviar el historial completo en cada llamada puede volverse costoso y exceder el límite de tokens del modelo en conversaciones muy largas.

En los próximos notebooks, exploraremos los **sistemas de memoria** que LangChain ofrece para gestionar estas limitaciones, como la memoria de búfer (`BufferMemory`) que automatiza la gestión del historial y la memoria de resumen (`SummaryMemory`) que condensa conversaciones largas para ahorrar tokens.